# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [220]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [221]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [222]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
        

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [223]:
# Make a connection to a Cassandra instance your local machine 

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
    session = cluster.connect()

except Exception as e:
    print(e)

#### Create Keyspace

In [224]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project1 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
    
except Exception as e:
    print(e)

#### Set Keyspace

In [225]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('project1')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [226]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# The primary key (sessionId, itemInSession) gives unique record in table artist_summary1

query = "CREATE TABLE IF NOT EXISTS artist_summary1"
query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text,\
 location text, sessionId int, song text, userId int, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)            

In [227]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_summary1 (artist, firstName, gender, itemInSession, lastName, length, level,\
                 location, sessionId, song, userId)"
    
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                

        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], \
                                int(line[8]), line[9], int(line[10])))
        
        
        
query = "select * from artist_summary1  WHERE sessionId = 338 AND itemInSession = 4;"


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Do a SELECT to verify that the data have been inserted into each table

In [228]:
## Add in the SELECT statement to verify the data was entered into the table

query = "select * from artist_summary1  where sessionId = 338;"


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession, row.artist, row.song, row.length)


1 Pixies Build High 89.36444091796875
2 The Roots / Jack Davey Atonement 155.95057678222656
3 Mike And The Mechanics A Beggar On A Beach Of Gold 275.1211853027344
4 Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [229]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# The primary key (userId, sessionId, itemInSession ) gives unique record in table artist_summary2
# The clustering column itemInSession sort the output


query = "CREATE TABLE IF NOT EXISTS artist_summary2"
query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text,\
 location text, sessionId int, song text, userId int, PRIMARY KEY (userId, sessionId, itemInSession ))"
try:
    session.execute(query)
except Exception as e:
    print(e)    
                    

In [230]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_summary2 (artist, firstName, gender, itemInSession, lastName, length, level,\
                 location, sessionId, song, userId)"
    
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], \
                                int(line[8]), line[9], int(line[10])))
        
        
        
query = "select * from artist_summary2  WHERE userid = 10 AND sessionid = 182;"



try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [231]:
## Add in the SELECT statement to verify the data was entered into the table

query = "select * from artist_summary2  where userid = 10 AND sessionid = 182;"


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.iteminsession, row.artist, row.song, row.length)

0 Down To The Bone Keep On Keepin' On 333.7660827636719
1 Three Drives Greece 2000 411.6370849609375
2 Sebastien Tellier Kilometer 377.73016357421875
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 181.2109832763672


In [232]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# The primary key (song, firstName, sessionId, itemInSession) gives unique record in table artist_summary3
# The clustering column  sessionId, itemInSession have been added to make each record unique


query = "CREATE TABLE IF NOT EXISTS artist_summary3"
query = query + "(artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text,\
 location text, sessionId int, song text, userId int, PRIMARY KEY (song, firstName, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)    
                   
                    

In [233]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_summary3 (artist, firstName, gender, itemInSession, lastName, length, level,\
                 location, sessionId, song, userId)"
    
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (line[0], line[1], line[2], int(line[3]), line[4], float(line[5]), line[6], line[7], \
                                int(line[8]), line[9], int(line[10])))
    
    
query = "select * from artist_summary3  WHERE song = 'All Hands Against His Own' ;"


try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( row.firstname, row.lastname)

Jacqueline Lynch
Sara Johnson
Tegan Levine


In [234]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

query = "select * from artist_summary3  WHERE song = 'All Hands Against His Own';"

#query = "select * from artist_summary1;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname, row.song)

Jacqueline Lynch All Hands Against His Own
Sara Johnson All Hands Against His Own
Tegan Levine All Hands Against His Own


### Drop the tables before closing out the sessions

In [235]:
## TO-DO: Drop the table before closing out the sessions

In [236]:
query = "drop table artist_summary1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


query = "drop table artist_summary2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
query = "drop table artist_summary3"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [237]:
session.shutdown()
cluster.shutdown()